In [244]:
import copy
import random
import pandas as pd
import numpy as np

# Battleship

Rules:
1. Each player arranges ships according to fleet
2. Take turns firing a shot
3. Mark Hits and Misses
4. Call out when a ship has been sunk
5. Sink all to win

Ships:
1. Carrier - 5
2. Battleship - 4
3. Cruiser - 3
4. Submarine - 3
5. Destroyer -2

10 Rows x 10 Columns  

|   |   |   |   |   |   |   |   |   |   |
|---|---|---|---|---|---|---|---|---|---|
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |

##### Parameters

In [245]:
Enemy_Ships = [["Carrier", 5, []],
               ["Battleship", 4, []],
               ["Cruiser", 3, []],
               ["Submarine", 3, []],
               ["Destroyer", 2, []]]

##### Helper Functions

In [246]:
# Is current proposed ship allocation valid?

def ValidPlacement(Board, i, j, k, length):
    # Board = Dictionary for Board
    # i = row of ship
    # j = column of ship
    # k = orientation of ship (0 = hoz, 1 = vert)
    # length = length of ship
    
    #Horizontal Ship
    if (k == 0):
        if (j>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i][j+l]
            if(occupied == 0):
                return 1
            else:
                return 0
            
    #Vertical Ship
    if (k == 1):
        if (i>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i+l][j]
            if(occupied == 0):
                return 1
            else:
                return 0

In [247]:
def ValidLocations(Board, Ship_List, Occupied):
    #Wri
    #Board: dictionary for occupied spaces
    #Ship_List: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    #Returns: Occupied, a shadow of the board containing all occupied spots
    
    for s in range(len(Ship_List)):
        valid_placements = []
        Ship = Ship_List[s]
        length = Ship[1]

        for i in range(1,11):
            for j in range(1,11):
                for k in range(2):
                     if (ValidPlacement(Board,i,j,k,length)==1):
                            valid_placements.append([i,j,k])
                            Occupied[i-1][j-1] = 1
        Ship[2] = valid_placements
        
    return Occupied

In [248]:
def SpacesOccupied(length,location):
    #length = length of ship
    #location = [i,j,k]
    #Returns list of spaces occupied by ship
    
    occupied = []
    i,j,k = location
    
    if (k==0):
        for l in range(length):
            occupied.append((i,j+l))
    if (k==1):
        for l in range(length):
            occupied.append((i+l,j))
        
    return occupied

In [249]:
def RandomSample(Ship_List):
    # Ship_List = ["Ship_Name", length, locations]
    # Return: Data frame of board with 1s for ship positions
    #        or all 0s if not a valid arrangement
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    
    locations = []
    for s in range(len(Ship_List)):
        Ship = Ship_List[s]
        position = random.choice(Ship[2])
        spaces = SpacesOccupied(Ship[1],position)
        locations.extend(spaces)
    #Check if valid configuration
    if(len(locations) == len(set(locations))):
        for t in locations:
            PDF.loc[t] = 1
        return PDF
    else:
        return False        

In [250]:
def NSamples(N,M,Ship_List):
    #N = Number of samples wanted
    #M = Max number of iterations
    #Ship_List = ["Ship_Name", length, locations]
    #Returns: (DF, S) Data frame of accumulated possible locations and number of actual samples taken
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    n = 0
    m = 0
    while( (n<N) & (m<M) ):
        m = m + 1
        Result = RandomSample(Ship_List)
        if (type(Result) != bool):
            PDF = PDF + Result
            n = n+1
    
    return(PDF,n)

##### Initialize Board

In [251]:
Board = {}
for i in range(1,11):
    Board[i] = {}
    for j in range(1,11):
        Board[i][j] = 0 
        
# Board = np.zeros((10,10))

##### Example

In [252]:
Board[5][5] = 1
Board[5][6] = 1
Board[6][5] = 1
Board[6][6] = 1

Board[2][2] = 1
Board[3][3] = 1
Board[4][4] = 1
Board[5][1] = 1

Board[10][10] = 1
Board[8][3] = 1
Board[2][7] = 1
Board[10][9] = 1

In [253]:
DispBoard = np.zeros((10,10))

for i in range(1,11):
    for j in range(1,11):
        if Board[i][j] == 0:
            DispBoard[i-1][j-1] = 0
        else:
            DispBoard[i-1][j-1] = 1

In [254]:
Occupied = np.zeros((10,10))
Occupied = ValidLocations(Board, Enemy_Ships, Occupied)

In [255]:
Occupied

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.]])

In [256]:
PDF,n = NSamples(2000,6000,Enemy_Ships)

In [257]:
PDF

,1,2,3,4,5,6,7,8,9,10
1,217,245,366,477,498,517,442,467,346,241
2,160,0,115,239,292,238,0,259,298,301
3,179,165,0,179,352,442,424,537,483,415
4,157,333,137,0,201,281,473,559,556,508
5,0,393,205,193,0,0,352,422,504,498
6,245,528,261,306,0,0,417,424,472,469
7,340,554,370,631,472,456,681,555,486,436
8,300,323,0,361,336,414,601,548,408,318
9,346,434,351,594,552,531,567,509,337,228
10,302,373,390,535,534,420,372,247,0,0


In [258]:
PDF/n

,1,2,3,4,5,6,7,8,9,10
1,0.1085,0.1225,0.1830,0.2385,0.2490,0.2585,0.2210,0.2335,0.1730,0.1205
2,0.0800,0.0000,0.0575,0.1195,0.1460,0.1190,0.0000,0.1295,0.1490,0.1505
3,0.0895,0.0825,0.0000,0.0895,0.1760,0.2210,0.2120,0.2685,0.2415,0.2075
4,0.0785,0.1665,0.0685,0.0000,0.1005,0.1405,0.2365,0.2795,0.2780,0.2540
5,0.0000,0.1965,0.1025,0.0965,0.0000,0.0000,0.1760,0.2110,0.2520,0.2490
6,0.1225,0.2640,0.1305,0.1530,0.0000,0.0000,0.2085,0.2120,0.2360,0.2345
7,0.1700,0.2770,0.1850,0.3155,0.2360,0.2280,0.3405,0.2775,0.2430,0.2180
8,0.1500,0.1615,0.0000,0.1805,0.1680,0.2070,0.3005,0.2740,0.2040,0.1590
9,0.1730,0.2170,0.1755,0.2970,0.2760,0.2655,0.2835,0.2545,0.1685,0.1140
10,0.1510,0.1865,0.1950,0.2675,0.2670,0.2100,0.1860,0.1235,0.0000,0.0000


In [259]:
n

2000

In [260]:
a = np.array([[1,3,2],[8,7,9],[5,6,4]])
#print(a)
arr = np.where(a==a.max())
x = arr[0]
y = arr[1]
#print(a[x].flatten()[y])

Unsunk = 0

In [275]:
def ShipSearch(Board, Enemy_Ships):
    # ShipSearch = [Board, Enemy_Ships]
    # Board: dictionary for occupied spaces
    # Enemy_Ships: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    # Returns: x,y coordinates of chosen spot (on board)
    
    # What is a good choice for N and M in NSamples?    
    PDF,n = NSamples(2000,6000,Enemy_Ships)
    
    arr = np.where(PDF==max(PDF.max()))
    x = arr[0]+1
    y = arr[1]+1
    #because max PDF[x][y] corresponds to Board[x+1][y+1]
    
    return x[0],y[0]

In [276]:
np.max(PDF.max())

681

In [280]:
x,y = ShipSearch(Board,Enemy_Ships)

In [281]:
print(x)
print(y)
print(x[0])
print(y[0])
#print(Enemy_Ships)

[7]
[7]
7
7


In [285]:
def isHit(Board, Occupied, x, y):
    # isHit = [Board, Enemy_Ships, x, y]
    # Board: dictionary for occupied spaces
    # Occupied: shadow board containing all occupied spaces
    # x, y: target x-y coordinates on board
    # Returns: whether or not it's a hit!
    
    if Occupied[x,y]:
        print('hit!')
        return 1
    else:
        print('miss!')
        return 0

In [286]:
# print(Occupied.shape)
isHit(Board,Occupied,x,y)

(10, 10)
hit!


1

In [287]:
ShipQueue = []

In [292]:
 for i in range(len(Enemy_Ships)):
        print('\n')
        for j in range(len(Enemy_Ships[i])):
            print(Enemy_Ships[i][2][j])




[1, 1, 0]
[1, 2, 0]
[1, 3, 0]


[1, 1, 0]
[1, 1, 1]
[1, 2, 0]


[1, 1, 0]
[1, 1, 1]
[1, 2, 0]


[1, 1, 0]
[1, 1, 1]
[1, 2, 0]


[1, 1, 0]
[1, 1, 1]
[1, 2, 0]


In [288]:
def ShipSink(Board, Enemy_Ships,ShipQueue):
    # ShipSink = [Board, Enemy_Ships]
    # Board: dictionary for occupied spaces
    # Enemy_Ships: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    # Returns: x,y coordinates of chosen spot (on board)
    return 1

In [289]:
# if len(ShipQueue) == 0:
#     ShipSearch(Board,Enemy_Ships)
# else:
#     ShipSink(Board,Enemy_Ships)

### To Do

1. Code should be divided into "Ship Search" and "Ship Sink" methods
    * Ship Search - tries to find ship to sink (mostly written)
    * Ship Sink - tries to sink ship when one is found (need to write)
2. Find space with highest probability to shoot into for "Ship Search"
3. Write "Ship Sink" method
    * If enemy ship is hit, iterate through list of possible ships that were hit to find most likely direction to start testing
    * Edge cases very important. Possible to hit multiple ships in trying tosink one so "ship sunk" queue very important
    